In [1]:
#must1
import tensorflow as tf 
assert tf.__version__.startswith('2')
from tensorflow import keras
import numpy as np

2024-01-10 13:26:07.625989: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 13:26:08.562510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#must2
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.optimizers import RMSprop, Adam
from keras.preprocessing import image
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
data_path = "early_data/"

In [3]:
data_path = "severe_data/"

In [4]:
#must3
validation_ratio = 0.1
from keras.preprocessing.image import ImageDataGenerator
train_data_gen = ImageDataGenerator(rescale=1.0/255, validation_split=validation_ratio,zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='nearest')
train_generator = train_data_gen.flow_from_directory(data_path, target_size=(224,224), batch_size=8, class_mode='categorical', subset='training')
valid_generator = train_data_gen.flow_from_directory(data_path, target_size=(224,224), batch_size=8, class_mode='categorical', subset='validation')

Found 8217 images belonging to 5 classes.
Found 910 images belonging to 5 classes.


In [5]:
#must4
labels = {value: key for key, value in train_generator.class_indices.items()}
for key, value in labels.items():
    print(f"{key} : {value}")

0 : black_spot
1 : canker
2 : greening
3 : healthy
4 : melanose


In [6]:
#must5
from keras.regularizers import l2

In [7]:
#must6
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=np.sqrt(0.1), patience=5)

In [8]:
#must6.5
from keras.layers import GlobalAveragePooling2D, Input
from keras.models import Model

In [9]:
#must7
# Xây dựng mô hình
num_classes = 5
IMG_SIZE = 224
batch_size = 8
epochs = 30

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet.EfficientNetB4(weights=None, include_top=False, input_tensor=inputs) 

# Thêm một lớp GlobalAveragePooling2D để thay thế lớp Fully Connected cuối cùng
x = GlobalAveragePooling2D()(base_model.output)




# Thêm lớp Dense mới với số đơn vị đầu ra là num_classes và regularization L2
x = Dense(num_classes, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='sigmoid')(x)

# Tạo mô hình mới
model = Model(inputs=inputs, outputs=x)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

2024-01-10 13:26:33.662046: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9220 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:03:00.0, compute capability: 7.5


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 stem_conv_pad (ZeroPadding  (None, 225, 225, 3)          0         ['normalization[0][0]']   

In [10]:
#must8
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('train_logger2.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('train_logger2.csv', append=True, separator='\t')

checkpoint_filepath = "cp.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [11]:
#must9
import time
import csv

In [12]:
#must10
# Optimizer và loss function
from keras.losses import CategoricalCrossentropy
loss = CategoricalCrossentropy()
#loss = BinaryCrossentropy()
optimizer = Adam(learning_rate=0.001)
model.compile(loss=loss, optimizer=optimizer, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_generator.samples // batch_size
validation_steps = valid_generator.samples // batch_size

hist6 = model.fit(train_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=valid_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

Epoch 1/30


2024-01-10 13:27:13.780302: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-01-10 13:27:18.911561: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-01-10 13:27:19.720844: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7efed000d960 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-10 13:27:19.720888: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-01-10 13:27:19.729875: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-10 13:27:19.905049: I ./tensorflow/compiler/jit/device_compiler.h:186] C

1027/1027 [==============================] - ETA: 0s - loss: 1.0847 - accuracy: 0.6053 - recall: 0.8807 - precision: 0.3729
Epoch 1: saving model to cp.ckpt
Epoch 1 - Training Time: 204.08 seconds
1027/1027 [==============================] - 204s 127ms/step - loss: 1.0847 - accuracy: 0.6053 - recall: 0.8807 - precision: 0.3729 - val_loss: 0.9709 - val_accuracy: 0.7423 - val_recall: 0.9613 - val_precision: 0.4912 - time_per_epoch: 204.0808
Epoch 2/30
1027/1027 [==============================] - ETA: 0s - loss: 0.6160 - accuracy: 0.7933 - recall: 0.9621 - precision: 0.4465
Epoch 2: saving model to cp.ckpt
Epoch 2 - Training Time: 127.14 seconds
1027/1027 [==============================] - 127s 124ms/step - loss: 0.6160 - accuracy: 0.7933 - recall: 0.9621 - precision: 0.4465 - val_loss: 0.6627 - val_accuracy: 0.7290 - val_recall: 0.9757 - val_precision: 0.4530 - time_per_epoch: 127.1441
Epoch 3/30
1027/1027 [==============================] - ETA: 0s - loss: 0.4855 - accuracy: 0.8396 - rec

In [13]:

#must11
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(valid_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('Test_data_metrics.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('EfficientNetB5_SVM_grabcut.h5')

114/114 [==============================] - 11s 96ms/step - loss: 0.1560 - accuracy: 0.9571 - recall: 0.9989 - precision: 0.5768
Test_loss: 0.15601666271686554
Test_accuracy: 0.9571428298950195
Test_recall: 0.9989010691642761
Test_precision: 0.5767766237258911


/home/tower1/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
